In [1]:
# Transfer Learning : MobileNetV2

In [2]:
# required imports

from tensorflow.keras.layers import  Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

In [3]:
# loading MobileNetV2 model
mobile_net_v2 = MobileNetV2(input_shape=[224, 224] + [3], weights='imagenet', include_top=False)

# acquiring trained parameters
for layer in mobile_net_v2.layers:
    layer.trainable = False
  

In [4]:
# adding a flatten layer
model_mobile_net_v2 = Flatten()(mobile_net_v2.output)

#adding the output layer
prediction = Dense(211, activation='softmax')(model_mobile_net_v2)

# create a model object
model_mobile_net_v2 = Model(inputs=mobile_net_v2.input, outputs=prediction)

model_mobile_net_v2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [5]:
# compile the model

model_mobile_net_v2.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [6]:
# required imports

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [7]:
# using custom variations on training images as given in docs

data_gen_mobilenet_v2 = ImageDataGenerator(preprocessing_function=preprocess_input)

In [8]:
# extract training images from directory

training_set = data_gen_mobilenet_v2.flow_from_directory(
                directory = "Currency_Images/coins/data/train",
                class_mode = 'categorical',
                target_size = (224, 224),
                batch_size = 32,
                subset = 'training'
                )

Found 6413 images belonging to 211 classes.


In [9]:
# validation set images extracted from directory

validation_set = data_gen_mobilenet_v2.flow_from_directory(
        directory = 'Currency_Images/coins/data/validation',
        class_mode='categorical',
        target_size = (224, 224)
)

Found 844 images belonging to 211 classes.


In [10]:
# test images extracted from directory

test_set = data_gen_mobilenet_v2.flow_from_directory(
        directory = 'Currency_Images/coins/data/test',
        class_mode='categorical',
        target_size = (224, 224),
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [11]:
# model fit on the training set

history_mobilenet_v2 = model_mobile_net_v2.fit_generator(
                                        training_set,
                                        validation_data = validation_set,
                                        epochs = 10
                             )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
201/201 [==============================] - 324s 2s/step - loss: 12.7120 - accuracy: 0.5303 - val_loss: 9.7948 - val_accuracy: 0.6789
Epoch 2/5
201/201 [==============================] - 313s 2s/step - loss: 2.7819 - accuracy: 0.8771 - val_loss: 11.8746 - val_accuracy: 0.6919
Epoch 3/5
201/201 [==============================] - 307s 2s/step - loss: 2.2449 - accuracy: 0.9245 - val_loss: 11.8303 - val_accuracy: 0.7097
Epoch 4/5
201/201 [==============================] - 304s 2s/step - loss: 2.1761 - accuracy: 0.9301 - val_loss: 13.1923 - val_accuracy: 0.7002
Epoch 5/5
201/201 [==============================] - 305s 2s/step - loss: 2.2404 - accuracy: 0.9354 - val_loss: 13.9926 - val_accuracy: 0.7168


In [12]:
# final evaluation on the test set

score_mobilenet_v2 = model_mobile_net_v2.evaluate_generator(test_set, verbose=2)


Instructions for updating:
Please use Model.evaluate, which supports generators.
27/27 - 33s - loss: 10.9867 - accuracy: 0.7239


In [16]:
# test loss and test accuracy

print('MobileNetV2 Model : Test Loss is : ', score_mobilenet_v2[0])
print('MobileNetV2 Model : Test Accuracy is : ', score_mobilenet_v2[1])

MobileNetV2 Model : Test Loss is :  10.986675262451172
MobileNetV2 Model : Test Accuracy is :  0.7239336371421814


In [14]:
model_mobile_net_v2.save("MobileNetV2_model_for_currency_classification.h5")